In [1]:
import conf

import os
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor,FasterRCNN_ResNet50_FPN_Weights
from torchinfo import summary

import torch

import aml.model_using as model_using
import aml.support_func as support_func
import aml.time_mesuarment as time_mesuarment
import sys

import aml.managers as managers
import aml.img_processing as img_processing
import random
import numpy as np
import pprint
from torchinfo import summary
from aml.img_processing import *

from PIL import Image
import aml.models as models
import matplotlib.pyplot as plt
import data_manip as dm
from pprint import pprint as Print
from PIL import Image


In [2]:
# BoxLabelByImgURL = torch.load(conf.BoxesLabelsByPath_filename)
# # bbox = [x,y,w,h]
# keys_ = BoxLabelByImgURL.keys()

In [3]:
# BoxLabel = 0
# Img_ = 0
# for key_ in keys_:
#     BoxLabel = BoxLabelByImgURL[key_]
#     imgURL = key_
#     Img_= np.transpose(np.array(Image.open(imgURL)))
#     break
# Print(BoxLabel)

In [4]:
# ImgWithBox_ = Img_
# Print(Img_.shape)
# plt.imshow(np.transpose(InsertBoxesToPILImage(Img_,BoxLabel['bboxes'])))


In [5]:
BoxLabelByImgURL = torch.load(conf.BoxesLabelsByPath_filename)

In [22]:
models_manager = managers.ModelsManager()
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
num_classes = 81 # 80 classes + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.eval()
# get number of input features for the classifier
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [7]:
# Print(summary(model))
# Print(model)

In [23]:
imgs = []
targets = []
fps = []
for fp_ in BoxLabelByImgURL.keys():
    BoxLabel= BoxLabelByImgURL[fp_]
    fps.append(fp_)
    # Print(BoxLabel)
    imgs.append(Norm(NPtoTensorGradFalse(np.transpose(np.array(PILToRGB(Image.open(fp_)),dtype=np.float32)))))
    targets.append({
        'boxes':NPtoTensorGradFalse(xywh_to_xyxy(np.array(BoxLabel['bboxes']))),
        'labels':NPtoTensorGradFalse(np.array(BoxLabel['labels']))
    })
    break

In [24]:
out = model(imgs,targets)

In [29]:
# Print(torch.hub.list('pytorch/vision:v0.13.1'))
# Print(out)
predicted_boxes = out[0]['boxes']
Print(predicted_boxes)
# Print(predicted_boxes)
npimage =np.transpose(np.array(PILToRGB(Image.open(fps[0]))))
npimage = InsertBoxesToNpArrayXYXY(npimage,boxes = predicted_boxes.cpu().detach().numpy())
plt.imshow(np.transpose(npimage))


tensor([[  0.0000,  59.8178, 360.0000, 640.0000],
        [  0.0000,   0.0000, 360.0000, 640.0000]], grad_fn=<StackBackward0>)


IndexError: index 640 is out of bounds for axis 2 with size 360